In [121]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
from dateutil.parser import parse

#overarching idea: constant time feedback if no change
#interrupt if got change
#defining constants
polling_freq = datetime.timedelta(seconds = 15)

f = lambda s: datetime.datetime.strptime(s,'%Y-%m-%d-%H-%M-%S')
df = pd.read_csv(r'C:\Users\willi\Desktop\EE4211\dataport-export_gas_oct2015-mar2016.csv')
num_unique_ids = len(set(df.dataid))
print("There are", num_unique_ids, "housholds in this dataset")

<class 'pandas.core.frame.DataFrame'>
There are 157 housholds in this dataset


In [118]:
anomalous_readings_by_id = {}
huge_readings_by_id = {}
for item in set(df.dataid):
    print("Processing", item)
    house = df.loc[df['dataid']==item]
    
    metering = np.array(house['meter_value'])
    days = house['localminute']
    days_time = list(set(list(map(lambda x: parse(x).date(), house['localminute']))))
    #days_time = np.array(days_time)
    difference_in_time = np.diff(days_time) #find the number of seconds that passed between each poll
    difference_in_meter = np.diff(metering)
    all_neg_values = np.where(difference_in_meter < 0)
    if len(all_neg_values[0]) > 0:
        anomalous_readings_by_id[item] = all_neg_values
    mean_meter = np.mean(difference_in_meter)
    std_meter = np.std(difference_in_meter)
    huge_readings = 15*mean_meter
    print(huge_readings)
    all_huge_values = np.where(difference_in_meter >= huge_readings)
    if len(all_huge_values[0]) > 0:
        huge_readings_by_id[item] = all_huge_values

Processing 9729
19.3689320388
Processing 7682
89.8938053097
Processing 5636
82.4868035191
Processing 5129
42.9163879599
Processing 5131
17.4239431055
Processing 2575
169.393939394
Processing 6673
7.32815860844
Processing 1042
60.0783494385
Processing 1556
71.1737598265
Processing 2072
22.8650687972
Processing 5658
184.87804878
Processing 6685
2805.58441558
Processing 35
14.1799342937
Processing 9766
140.661990355
Processing 44
91.0465116279
Processing 2094
17.7452339688
Processing 3635
31.842133914
Processing 8244
55.6535947712
Processing 1589
19.9836818337
Processing 7739
39.2260830325
Processing 7741
48.793195109
Processing 1086
10.8708538697
Processing 9278
36.85546875
Processing 3134
58.812250996
Processing 4671
2221.5
Processing 5193
14.9483635616
Processing 77
8.1726268489
Processing 2638
60.5950500263
Processing 9295
14.7597331625
Processing 1103
299.525179856
Processing 2129
32.1804729436
Processing 1619
51.1871227364
Processing 2645
1714.10958904
Processing 94
9.79082952606
Pr

In [135]:
for key in anomalous_readings_by_id:
    house = df.loc[df['dataid']==key]
    anomalous_readings_index = anomalous_readings_by_id[key][0]
    #print(anomalous_readings_index)
    house.reset_index(inplace = True, drop = True)
    print("Malfunctioning Meter ID: ", key)
    for index in anomalous_readings_index:
        print('From', house.loc[index]['localminute'], 'to', house.loc[index+1]['localminute'])
        print('Negative reading of', house.loc[index]['meter_value'], 'to', house.loc[index+1]['meter_value'])
        print(-(house.loc[index]['meter_value'] - house.loc[index+1]['meter_value']))
        print('........................................')
    print('\n')

Malfunctioning Meter ID:  5129
From 2015-10-17 18:36:59-05 to 2015-10-17 18:39:09-05
Negative reading of 153964 to 153960
-4
........................................
From 2015-10-17 22:39:55-05 to 2015-10-17 22:40:20-05
Negative reading of 153966 to 153964
-2
........................................
From 2015-10-20 12:32:32-05 to 2015-10-20 13:02:24-05
Negative reading of 154032 to 154030
-2
........................................
From 2015-12-07 16:34:10.272889-06 to 2015-12-07 17:10:13-06
Negative reading of 168484 to 156770
-11714
........................................
From 2015-12-07 18:31:11.38745-06 to 2015-12-07 18:39:35-06
Negative reading of 168484 to 156772
-11712
........................................
From 2015-12-07 19:24:03.933284-06 to 2015-12-07 19:26:21-06
Negative reading of 168486 to 156772
-11714
........................................
From 2015-12-07 20:25:08.476761-06 to 2015-12-07 21:13:13-06
Negative reading of 168486 to 156786
-11700
......................

Negative reading of 162362 to 144754
-17608
........................................
From 2015-12-08 23:14:14.926004-06 to 2015-12-08 23:14:56-06
Negative reading of 162362 to 144754
-17608
........................................
From 2015-12-09 01:30:13.887505-06 to 2015-12-09 01:34:09-06
Negative reading of 162364 to 144754
-17610
........................................
From 2015-12-09 01:43:07.877083-06 to 2015-12-09 01:50:06-06
Negative reading of 162364 to 144754
-17610
........................................
From 2015-12-09 02:46:07.857588-06 to 2015-12-09 02:57:58-06
Negative reading of 162364 to 144756
-17608
........................................
From 2015-12-09 05:13:15.797503-06 to 2015-12-09 05:21:00-06
Negative reading of 162366 to 144756
-17610
........................................
From 2015-12-09 05:47:11.786024-06 to 2015-12-09 06:06:01-06
Negative reading of 162366 to 144768
-17598
........................................
From 2015-12-09 08:44:21.210749-06 to 2

Negative reading of 106096 to 94476
-11620
........................................
From 2015-12-11 08:28:50.453367-06 to 2015-12-11 09:27:41-06
Negative reading of 106098 to 94476
-11622
........................................
From 2015-12-11 09:35:45.925517-06 to 2015-12-11 09:36:44-06
Negative reading of 106098 to 94476
-11622
........................................
From 2015-12-11 13:00:48.354957-06 to 2015-12-11 14:14:50-06
Negative reading of 106098 to 94478
-11620
........................................
From 2015-12-11 15:14:30.322221-06 to 2015-12-11 15:56:37-06
Negative reading of 106100 to 94478
-11622
........................................
From 2015-12-11 16:16:42.826367-06 to 2015-12-11 16:46:31-06
Negative reading of 106100 to 94478
-11622
........................................
From 2015-12-11 18:17:43.346311-06 to 2015-12-11 18:33:47-06
Negative reading of 106100 to 94492
-11608
........................................
From 2015-12-11 19:26:30.354703-06 to 2015-12-

From 2015-12-09 21:12:10.216846-06 to 2015-12-09 22:12:03-06
Negative reading of 128754 to 116252
-12502
........................................
From 2015-12-09 22:39:11.685224-06 to 2015-12-09 22:52:14-06
Negative reading of 128756 to 116252
-12504
........................................
From 2015-12-09 23:24:17.168343-06 to 2015-12-09 23:30:24-06
Negative reading of 128756 to 116254
-12502
........................................
From 2015-12-09 23:41:08.160869-06 to 2015-12-09 23:48:27-06
Negative reading of 128756 to 116254
-12502
........................................
From 2015-12-09 23:59:08.15359-06 to 2015-12-10 00:09:19-06
Negative reading of 128758 to 116254
-12504
........................................
From 2015-12-10 01:03:01.123501-06 to 2015-12-10 01:06:13-06
Negative reading of 128758 to 116256
-12502
........................................
From 2015-12-10 02:03:14.095779-06 to 2015-12-10 02:04:04-06
Negative reading of 128760 to 116258
-12502
....................

Negative reading of 166228 to 146610
-19618
........................................
From 2015-12-11 05:56:18.435134-06 to 2015-12-11 06:01:13-06
Negative reading of 166232 to 146612
-19620
........................................
From 2015-12-11 06:25:16.941711-06 to 2015-12-11 06:46:20-06
Negative reading of 166232 to 146614
-19618
........................................
From 2015-12-11 09:36:27.462108-06 to 2015-12-11 09:49:25-06
Negative reading of 166236 to 146618
-19618
........................................
From 2015-12-11 09:55:12.970578-06 to 2015-12-11 10:07:18-06
Negative reading of 166236 to 146618
-19618
........................................
From 2015-12-11 12:22:30.994145-06 to 2015-12-11 12:23:07-06
Negative reading of 166240 to 146620
-19620
........................................
From 2015-12-11 12:36:30.993576-06 to 2015-12-11 12:44:05-06
Negative reading of 166240 to 146622
-19618
........................................
From 2015-12-11 13:37:23.508534-06 to 2

From 2015-12-08 04:32:03.56972-06 to 2015-12-08 04:43:54-06
Negative reading of 416036 to 389086
-26950
........................................
From 2015-12-08 05:06:08.589509-06 to 2015-12-08 05:08:00-06
Negative reading of 416038 to 389092
-26946
........................................
From 2015-12-08 05:34:03.101668-06 to 2015-12-08 05:37:06-06
Negative reading of 416038 to 389094
-26944
........................................
From 2015-12-08 06:04:57.614493-06 to 2015-12-08 06:30:18-06
Negative reading of 416038 to 389140
-26898
........................................
From 2015-12-08 06:30:57.129479-06 to 2015-12-08 06:34:20-06
Negative reading of 416040 to 389142
-26898
........................................
From 2015-12-08 06:58:55.14315-06 to 2015-12-08 07:33:10-06
Negative reading of 416040 to 389154
-26886
........................................
From 2015-12-08 08:10:56.67876-06 to 2015-12-08 08:11:04-06
Negative reading of 416042 to 389166
-26876
......................

Negative reading of 273216 to 239610
-33606
........................................
From 2015-12-09 23:51:01.011002-06 to 2015-12-10 00:23:05-06
Negative reading of 273216 to 239612
-33604
........................................
From 2015-12-10 00:53:56.542615-06 to 2015-12-10 00:55:11-06
Negative reading of 273218 to 239614
-33604
........................................
From 2015-12-10 02:09:50.582665-06 to 2015-12-10 02:48:11-06
Negative reading of 273218 to 239624
-33594
........................................
From 2015-12-10 03:23:53.614685-06 to 2015-12-10 03:26:03-06
Negative reading of 273218 to 239632
-33586
........................................
From 2015-12-10 03:52:58.628315-06 to 2015-12-10 03:58:56-06
Negative reading of 273218 to 239632
-33586
........................................
From 2015-12-10 04:42:56.652013-06 to 2015-12-10 05:23:14-06
Negative reading of 273218 to 239648
-33570
........................................
From 2015-12-10 06:08:53.189078-06 to 2

From 2015-12-09 07:05:01.56078-06 to 2015-12-09 07:10:05-06
Negative reading of 233418 to 214740
-18678
........................................
From 2015-12-09 07:12:58.055989-06 to 2015-12-09 07:18:08-06
Negative reading of 233418 to 214742
-18676
........................................
From 2015-12-09 07:48:01.047271-06 to 2015-12-09 08:18:12-06
Negative reading of 233418 to 214752
-18666
........................................
From 2015-12-09 08:33:54.530608-06 to 2015-12-09 09:00:07-06
Negative reading of 233418 to 214754
-18664
........................................
From 2015-12-09 10:49:07.488878-06 to 2015-12-09 11:50:13-06
Negative reading of 233420 to 214766
-18654
........................................
From 2015-12-09 12:27:49.485274-06 to 2015-12-09 12:47:10-06
Negative reading of 233420 to 214766
-18654
........................................
From 2015-12-09 14:01:57.48173-06 to 2015-12-09 14:06:04-06
Negative reading of 233422 to 214768
-18654
.....................

From 2015-12-09 20:22:39.560261-06 to 2015-12-09 20:23:44-06
Negative reading of 274200 to 256542
-17658
........................................
From 2015-12-09 21:39:39.57196-06 to 2015-12-09 21:50:43-06
Negative reading of 274202 to 256566
-17636
........................................
From 2015-12-09 22:08:40.576938-06 to 2015-12-09 22:11:44-06
Negative reading of 274202 to 256570
-17632
........................................
From 2015-12-09 22:42:39.579484-06 to 2015-12-09 22:43:43-06
Negative reading of 274204 to 256570
-17634
........................................
From 2015-12-09 22:50:39.579777-06 to 2015-12-09 23:03:43-06
Negative reading of 274204 to 256570
-17634
........................................
From 2015-12-09 23:38:40.58108-06 to 2015-12-09 23:56:43-06
Negative reading of 274204 to 256570
-17634
........................................
From 2015-12-10 00:43:40.082087-06 to 2015-12-10 00:50:43-06
Negative reading of 274204 to 256570
-17634
.....................

In [120]:
for key in huge_readings_by_id:
    house = df.loc[df['dataid']==key]
    huge_readings_index = huge_readings_by_id[key][0]
    print(huge_readings_index)
    house.reset_index(inplace = True, drop = True)
    for index in huge_readings_index:
        print(house.loc[index-1])
        print(house.loc[index])
        print(house.loc[index+1])
        print('........................................')

[ 1720  3626  3628  3964  4046  4047  4056  4066  4068  4096  4156  4244
  4402  4472  4475  4595  5042  5217  5218  5246  5277  5313  5317  5318
  5844  5845  5910  5929  6002  6071  6082  6138  6162  6187  6224  6234
  6291  6300  6303  6380  6435  6447  6448  6450  6518  6553  6584  6586
  6664  6796  6812  6817  6825  6827  6863  6874  6906  6908  6959  6960
  6965  6966  6970  7048  7112  7322  7331  7338  7340  7430  7431  7501
  7571  7572  7644  7696  7698  7712  7713  7779  7786  7844  7848  7850
  7854  8064  8065  8066  8110  8119  8120  8124  8125  8127  8189  8190
  8192  8545  8575  8576  8577  8622  8627  8656  8748  8749  8751  8755
  8828  8832  8833  8878  8882  8884  9482  9897  9900  9959 10091 11572
 11642 11647 11716]
localminute    2015-10-25 23:44:14-05
dataid                           9729
meter_value                    122766
Name: 1719, dtype: object
localminute    2015-10-25 23:45:13-05
dataid                           9729
meter_value                    122

Name: 2099, dtype: object
localminute    2016-01-17 23:24:04.20468-06
dataid                                 5636
meter_value                          232376
Name: 2100, dtype: object
localminute    2016-01-18 05:44:03.971706-06
dataid                                  5636
meter_value                           232488
Name: 2101, dtype: object
........................................
[1800 1929 1932 2061 2074 2205 2209 2212 2215 2221 2226 2231 2234 2237 2242
 2245 2250 2253 2258 2264 2268 2277 2281 2283 2286 2293 2297 2301 2307 2309
 2313 2317 2321 2325 2329 2339 2342 2347 2354 2357 2362 2365 2370 2376 2380
 2385 2388 2391 2393 2396 2404 2407 2410 2412 2416 2419 2423 2429 2433 2438
 2442 2444 2446 2450 2454 2457 2461 2466 2469 2472 2475 2480 2483 2489 2494
 2497 2503 2514 2653 2698 2710 3052 3092 3269 3298 3306 3313 3353 3385 3404
 3412 3419 3421 3428 3573 3582 3619 3623 3624 3671 3716 3724 3729 3737 3755
 3861 3884 3888 3891 3940 3964 4126 4139 4144 4157 4360]
localminute    2015-11-22

Name: 9826, dtype: object
........................................
localminute    2016-01-27 01:18:45.933256-06
dataid                                  5131
meter_value                           116406
Name: 9844, dtype: object
localminute    2016-01-27 01:18:45.937939-06
dataid                                  5131
meter_value                           116406
Name: 9845, dtype: object
localminute    2016-01-27 02:17:58.910285-06
dataid                                  5131
meter_value                           116428
Name: 9846, dtype: object
........................................
localminute    2016-01-27 06:00:52.32454-06
dataid                                 5131
meter_value                          116498
Name: 9860, dtype: object
localminute    2016-01-27 06:50:02.307458-06
dataid                                  5131
meter_value                           116502
Name: 9861, dtype: object
localminute    2016-01-27 07:50:54.2902-06
dataid                                5131
mete

........................................
localminute    2016-02-07 05:05:13.859009-06
dataid                                  2072
meter_value                           284978
Name: 8955, dtype: object
localminute    2016-02-07 05:09:15.858733-06
dataid                                  2072
meter_value                           284984
Name: 8956, dtype: object
localminute    2016-02-07 05:26:09.857459-06
dataid                                  2072
meter_value                           285020
Name: 8957, dtype: object
........................................
localminute    2016-02-08 05:08:20.550314-06
dataid                                  2072
meter_value                           285186
Name: 9052, dtype: object
localminute    2016-02-08 05:09:20.549764-06
dataid                                  2072
meter_value                           285186
Name: 9053, dtype: object
localminute    2016-02-08 06:07:14.023274-06
dataid                                  2072
meter_value            

KeyError: 'the label [-1] is not in the [index]'

In [177]:
len(anomalous_readings_by_id)

43

In [119]:
len(huge_readings_by_id)

145

In [175]:
#house = df.loc[df['dataid']==739]

#parse(house['localminute'][0]).date()
#days_739 = list(set(list(map(lambda x: parse(x).date(), house['localminute']))))
#days_739.sort()
#days_739 = np.array(days_739)
#diff_days = np.diff(days_739)
#np.where(diff_days > datetime.timedelta(1))

#print(np.diff(days_739))
malfunction_meters_by_d_missing = []
for key in huge_readings_by_id:
    house = df.loc[df['dataid']==key]
    days_sequence = list(map(lambda x: parse(x).date(), house['localminute']))
    #days_sequence.sort()
    days_sequence = np.array(days_sequence)
    diff_days = np.diff(days_sequence)
    missing_index = np.where(diff_days > datetime.timedelta(1))
    print(missing_index[0])
    house.reset_index(inplace = True, drop = True)
    if len(missing_index[0])> 0:
        malfunction_meters_by_d_missing.append(key)
        print("Meter", key, "is missing days")
        print("++++++++++++++++++++++++++++++++++++++++")
        for index in missing_index[0]:
            print(house.loc[index])
            print(house.loc[index+1])
        print("\n")

[1720 5501]
Meter 9729 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 23:45:13-05
dataid                           9729
meter_value                    122766
Name: 1720, dtype: object
localminute    2015-10-27 00:22:00-05
dataid                           9729
meter_value                    122792
Name: 1721, dtype: object
localminute    2015-12-21 23:37:52-06
dataid                           9729
meter_value                    126748
Name: 5501, dtype: object
localminute    2015-12-23 00:04:44-06
dataid                           9729
meter_value                    126758
Name: 5502, dtype: object


[ 477 3218]
Meter 7682 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 23:18:57-05
dataid                           7682
meter_value                    229782
Name: 477, dtype: object
localminute    2015-10-27 00:12:50-05
dataid                           7682
meter_value                    229860
Name: 478, dtype: object


[1305]
Meter 35 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 23:35:13-05
dataid                             35
meter_value                     94208
Name: 1305, dtype: object
localminute    2015-10-27 00:10:09-05
dataid                             35
meter_value                     94256
Name: 1306, dtype: object


[  28   43  286  299  823 2077]
Meter 9766 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-09 01:05:57-05
dataid                           9766
meter_value                    158776
Name: 28, dtype: object
localminute    2015-10-11 16:24:53-05
dataid                           9766
meter_value                    158886
Name: 29, dtype: object
localminute    2015-10-13 22:39:18-05
dataid                           9766
meter_value                    159022
Name: 43, dtype: object
localminute    2015-10-15 02:33:57-05
dataid                           9766
meter_value                    159066
Name: 44, dtype: 

[ 3980 18693]
Meter 1589 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 23:56:27-05
dataid                           1589
meter_value                    194690
Name: 3980, dtype: object
localminute    2015-10-27 00:02:55-05
dataid                           1589
meter_value                    194726
Name: 3981, dtype: object
localminute    2016-02-11 23:54:57.058817-06
dataid                                  1589
meter_value                           223690
Name: 18693, dtype: object
localminute    2016-02-13 00:01:40.264122-06
dataid                                  1589
meter_value                           223844
Name: 18694, dtype: object


[ 839 4288 4326]
Meter 7739 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 23:22:34-05
dataid                           7739
meter_value                    150018
Name: 839, dtype: object
localminute    2015-10-27 00:08:37-05
dataid                           7739
meter_value 

[4076]
Meter 9295 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 23:23:41-05
dataid                           9295
meter_value                    159062
Name: 4076, dtype: object
localminute    2015-10-27 00:01:26-05
dataid                           9295
meter_value                    159152
Name: 4077, dtype: object


[ 20  46  50  54  58  68 110 112 167 297 304]
Meter 1103 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-31 22:12:00-05
dataid                           1103
meter_value                    183210
Name: 20, dtype: object
localminute    2015-11-02 05:28:00-06
dataid                           1103
meter_value                    183224
Name: 21, dtype: object
localminute    2015-11-07 09:51:59-06
dataid                           1103
meter_value                    183302
Name: 46, dtype: object
localminute    2015-11-15 13:23:57-06
dataid                           1103
meter_value                    183392
N

[10040]
Meter 94 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-11-11 23:57:06-06
dataid                             94
meter_value                    117974
Name: 10040, dtype: object
localminute    2015-11-13 00:20:26-06
dataid                             94
meter_value                    118008
Name: 10041, dtype: object


[ 165  292  439  462  467  549  795  916  926 1011 1016 1017]
Meter 4193 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 21:36:20-05
dataid                           4193
meter_value                    290262
Name: 165, dtype: object
localminute    2015-10-27 00:46:19-05
dataid                           4193
meter_value                    290274
Name: 166, dtype: object
localminute    2015-11-15 15:13:13-06
dataid                           4193
meter_value                    290706
Name: 292, dtype: object
localminute    2015-11-17 13:19:13-06
dataid                           4193
meter_value        

...

[5230]
Meter 871 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 23:54:41-05
dataid                            871
meter_value                    106554
Name: 5230, dtype: object
localminute    2015-10-27 00:01:53-05
dataid                            871
meter_value                    106566
Name: 5231, dtype: object


[]
[4082]
Meter 7017 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 23:57:11-05
dataid                           7017
meter_value                    376720
Name: 4082, dtype: object
localminute    2015-10-27 00:09:41-05
dataid                           7017
meter_value                    376774
Name: 4083, dtype: object


[ 126  190  266  291  303  305  327 1432 1444 1520 1521 1522 1523 1529 1532
 1534 1535 1536 1542 1543 1724]
Meter 6505 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-20 14:52:16-05
dataid                           6505
meter_value                    1

[2689]
Meter 7030 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 23:47:25-05
dataid                           7030
meter_value                    142742
Name: 2689, dtype: object
localminute    2015-10-27 00:11:30-05
dataid                           7030
meter_value                    142796
Name: 2690, dtype: object


[  2   4   6   7   8  28  29  45  49  50  51 102 443]
Meter 8059 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-08 23:36:28-05
dataid                           8059
meter_value                    101636
Name: 2, dtype: object
localminute    2015-10-13 16:37:27-05
dataid                           8059
meter_value                    101764
Name: 3, dtype: object
localminute    2015-10-14 05:26:27-05
dataid                           8059
meter_value                    101766
Name: 4, dtype: object
localminute    2015-10-18 09:40:25-05
dataid                           8059
meter_value                    101

[684]
Meter 2449 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 23:47:23-05
dataid                           2449
meter_value                    172162
Name: 684, dtype: object
localminute    2015-10-27 00:20:23-05
dataid                           2449
meter_value                    172222
Name: 685, dtype: object


[524]
Meter 8084 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 23:51:32-05
dataid                           8084
meter_value                     92302
Name: 524, dtype: object
localminute    2015-10-27 00:24:22-05
dataid                           8084
meter_value                     92318
Name: 525, dtype: object


[   8  227 4993 4999 5012]
Meter 2965 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-02 20:22:14-05
dataid                           2965
meter_value                    191512
Name: 8, dtype: object
localminute    2015-10-27 00:37:19-05
dataid                

[13794]
Meter 9639 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2016-03-22 12:25:10.992863-05
dataid                                  9639
meter_value                           254410
Name: 13794, dtype: object
localminute    2016-03-28 09:52:54.831078-05
dataid                                  9639
meter_value                           255418
Name: 13795, dtype: object


[2084]
Meter 9134 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 23:41:07-05
dataid                           9134
meter_value                    231012
Name: 2084, dtype: object
localminute    2015-10-27 00:03:12-05
dataid                           9134
meter_value                    231086
Name: 2085, dtype: object


[  0   4   6  37 639 925 926]
Meter 6578 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-03 04:51:53-05
dataid                           6578
meter_value                    157806
Name: 0, dtype: object
localm

[3381]
Meter 483 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 23:45:05-05
dataid                            483
meter_value                    361168
Name: 3381, dtype: object
localminute    2015-10-27 00:04:19-05
dataid                            483
meter_value                    361180
Name: 3382, dtype: object


[11629]
Meter 2034 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 23:57:08-05
dataid                           2034
meter_value                     72912
Name: 11629, dtype: object
localminute    2015-10-27 00:00:22-05
dataid                           2034
meter_value                     72936
Name: 11630, dtype: object


[517]
Meter 3577 is missing days
++++++++++++++++++++++++++++++++++++++++
localminute    2015-10-25 22:12:16-05
dataid                           3577
meter_value                    172482
Name: 517, dtype: object
localminute    2015-10-27 00:26:11-05
dataid                           

In [176]:
for key in anomalous_readings_by_id:
    if key not in malfunction_meters_by_d_missing:
        malfunction_meters_by_d_missing.append(key)
print(len(malfunction_meters_by_d_missing))
print(malfunction_meters_by_d_missing)

136
[9729, 7682, 5636, 5129, 5131, 2575, 6673, 1042, 1556, 2072, 5658, 6685, 35, 9766, 44, 2094, 3635, 1589, 7739, 7741, 1086, 9278, 3134, 5193, 77, 2638, 9295, 1103, 2129, 1619, 2645, 94, 4193, 7794, 114, 7287, 9849, 4732, 8829, 4228, 3723, 661, 5785, 5275, 4767, 1697, 1185, 6830, 5810, 1714, 6836, 1718, 5814, 2233, 8890, 187, 3778, 8386, 4296, 6863, 7900, 222, 739, 9956, 744, 7919, 252, 1791, 4352, 1792, 9474, 1283, 9982, 7429, 1790, 5892, 1800, 1801, 3849, 4356, 6412, 8467, 5395, 4373, 5403, 7965, 2335, 7460, 3367, 3893, 5439, 4421, 2378, 5972, 9052, 4447, 2818, 871, 7017, 6505, 5484, 370, 7030, 8059, 4998, 1415, 2449, 8084, 2965, 8086, 2461, 9631, 9121, 4514, 2980, 2470, 9639, 9134, 6578, 4029, 4031, 3527, 7117, 3544, 8155, 8156, 3039, 2018, 1507, 484, 483, 2034, 3577, 7674, 8703, 7989]


In [242]:
from datetime import datetime, date, timedelta
df['localminute'] = df['localminute'].map(lambda x: str(x)[:-3])
df['localminute'] = pd.to_datetime(df['localminute'], infer_datetime_format=True)
df_oct = df[df['localminute']<=datetime(year=2015, month=10, day=31, hour=23, minute=59, second=59)]
df_oct_by_id = df_oct.sort_values(by=['dataid', 'localminute'])
df_oct_by_id.reset_index(inplace = True, drop = True)
start_hour = datetime(year=2015, month=10, day=1, hour=0, minute=0, second=0)
start_id = df_oct_by_id.loc[0]['dataid']
print(start_id)

all_hourly_readings_by_id = {}
for item in list(set(df['dataid'])):
    all_hourly_readings_by_id[item] = []

print(len(all_hourly_readings_by_id))
average = 0
curr_day = df_oct_by_id[(df_oct_by_id['localminute'] >= start_hour) & (df_oct_by_id['localminute'] < start_hour+timedelta(hours=1))]
len(list(set(df_oct_by_id['dataid'])))
curr_day[curr_day['dataid']==35]['meter_value'].mean()
while start_hour + timedelta(hours=1) < datetime(year=2015, month=11, day=1, hour=0, minute=0, second=0):
    curr_day = df_oct_by_id[(df_oct_by_id['localminute'] >= start_hour) & (df_oct_by_id['localminute'] < start_hour+timedelta(hours=1))]
    measured_keys = list(set(df_oct_by_id['dataid']))
    for key in all_hourly_readings_by_id:
        if key not in measured_keys:
            if len(all_hourly_readings_by_id[key]) == 0:
                all_hourly_readings_by_id[key].append(0)
            else:
                all_hourly_readings_by_id[key].append(all_hourly_readings_by_id[key][-1])
        else:
            mean_data = curr_day[curr_day['dataid']==key]['meter_value'].mean()
            all_hourly_readings_by_id[key].append(mean_data)
    start_hour = start_hour + timedelta(hours=1)

print(all_hourly_readings_by_id)
#for item in df_oct_by_id:
 #   print(item)
 #   break
#print(start_hour+timedelta(hours=1))
#df_oct_hour = df_oct.set_index('localminute').groupby('dataid').resample('H').fillna(method='ffill')
#print(type(df_oct_hour))
#print(df_oct_hour)

35
157
{9729: [122186.0, 122186.0, 122186.0, 122187.5, 122188.0, 122188.0, 122188.0, 122188.5, 122193.0, 122196.0, 122196.0, 122197.33333333333, 122198.0, 122198.0, 122198.0, 122198.0, nan, 122200.0, 122200.0, 122200.0, 122200.0, 122201.0, 122202.0, 122202.0, 122202.0, 122202.0, 122203.0, 122204.0, 122204.0, 122204.0, 122204.0, 122209.66666666667, 122216.0, 122216.0, nan, 122218.0, nan, 122218.0, 122218.66666666667, 122220.0, 122220.0, 122220.0, 122222.0, 122224.0, 122224.0, 122224.0, 122224.0, 122224.0, 122226.0, 122226.0, 122226.0, 122226.0, 122226.0, 122228.0, 122228.0, 122228.0, nan, 122230.0, 122230.0, 122230.0, 122231.6, 122232.0, 122232.0, 122232.0, 122233.33333333333, 122240.0, 122240.0, 122242.0, 122242.0, 122242.0, 122242.0, 122243.0, 122244.0, 122244.0, 122250.0, 122250.0, 122250.0, 122250.0, 122250.0, 122252.0, 122252.0, 122258.0, 122262.0, 122262.0, 122262.4, 122264.0, 122264.0, 122264.0, 122268.0, 122272.0, 122280.4, nan, 122282.0, nan, 122284.0, 122284.0, nan, 122284.0, 